### Match US News ranking with colleges

In [3]:
import pandas as pd
import numpy as np
from ggplot import *
%matplotlib inline

In [4]:
interns = pd.read_csv('../proc_data/Interns_combined.csv')
usnews = pd.read_csv('../proc_data/ranking_university_USnews.csv', header=None)

In [5]:
v = interns.College.value_counts()

In [6]:
usnews.columns = ['Position','Rank','School','Location']

In [7]:
cols = []
for school in v.keys():
    if school.lower() in usnews.School.str.lower().values:
        try:
            cols.append(usnews[usnews.School == school].Position.values[0])
        except:
            cols.append(school)
    else:
        cols.append(school)

In [9]:
f = zip(cols, v.values)
df = pd.DataFrame({'School':list(v.keys()), 'Rank':cols, 'Count':v.values})
#df.to_csv('Manual_Fill_School_Rank.csv')

In [10]:
q = sorted(f, key=lambda x: x[0])
q2 = [x for x in q if isinstance(x[0], int)]

In [11]:
x, y = zip(*q2)
df = pd.DataFrame({'Rank':x, 'Count':y})

In [16]:
#4, 7, 10, 13, 14, 17-19
#columbia, MIT, Caltech


### Clean College State data for for R

In [58]:
st_dict = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}

In [92]:
df = pd.read_csv('../proc_data/Interns_combined.csv')
college_state = df.College_State.value_counts()
cdict = dict(zip(college_state.keys().str.strip(), college_state.values))

cdict['DC'] += cdict['D.C.']
del cdict['D.C.']

#Split DC's students equally into MD and VA
cdict['MD'] += cdict['DC']/2
cdict['VA'] += cdict['DC']/2

#
d={}
for item in st_dict.iteritems():
    if item[0] in cdict:
        d[item[1].lower()] = cdict[item[0]]
    else:
        d[item[1].lower()] = 0
newdf = pd.DataFrame.from_dict(d, orient='index')
newdf.columns = ['Count']
newdf.reset_index(level=0, inplace=True)
newdf.columns = ['State', 'Interns']
nonstates = ['american samoa', 'national', 'northern mariana islands', 'puerto rico', 'guam', 'district of columbia','virgin islands']
newdf = newdf[ ~newdf.State.isin(nonstates)]

In [98]:
newdf.to_csv('../proc_data/state_intern.csv', index=False)

### Clean college enrollment data for R

In [168]:
df = pd.read_csv('../raw_data/2008_college_state_enrollment.csv', header=None)
df.columns = ['State', 'College_Enr']
count = df.College_Enr.apply(lambda x: int("".join( [c for c in x if str.isalnum(c)] )))
df['College_Enr'] = count
df['State'] = df.State.apply(lambda x: x.lower())
df.drop(df[df.State=='district of columbia'].index, inplace=True)
total = sum(df.College_Enr)
df['College_Frct'] = df.College_Enr.apply(lambda x: float(x)/total * 100)

In [169]:
interns = pd.read_csv('../proc_data/state_intern.csv')
total = sum(interns.Interns)
interns['Intern_Frct'] = interns.Interns.apply(lambda x: float(x)/total*100)
newdf = df.merge(interns,how='left', on='State')
newdf['Diff'] = newdf.Intern_Frct - newdf.College_Frct

In [170]:
newdf.to_csv('../proc_data/state_intern_fraction.csv', index=False)

### How many interns went to serve in the WH?

In [13]:
whemp = pd.read_csv('../../whitehouse_pay/raw_data/combined_wh_pay.csv')


In [32]:
a = set(whemp.Cleaned_Name.values)
b = set(interns.Cleaned_Names.values)
namelist = a&b

In [54]:
cont = interns[ interns.Cleaned_Names.isin(namelist)]
total = len(interns[ (interns.Year == 2013) & (interns.Season=='summer')])
print 'total 2013 summer:', total
print 'intern to employee:', len(namelist)-1
print 'percentage', (len(namelist)-1)/float(total)*100

total 2013 summer: 147
intern to employee: 10
percentage 6.80272108844
